# Physics-Informed Neural Network (PINN) for Granular Segregation - Phase 2.2

This notebook implements a PINN for inverse modeling of the non-dimensionalized segregation PDE.

## Phase 2.2: Neural Network for Segregation Velocity

In this phase, we replace the functional form of the segregation velocity $\Lambda(1-\tilde{x})g(\tilde{z})(1-c)$ with a **trainable neural network**. The segregation term in the PDE is parameterized as $\partial/\partial\tilde{z}[v_{seg}(\tilde{x}, \tilde{z}, \tilde{t}, \dot{\gamma}, c) \cdot c]$, where $v_{seg}$ is parameterized by a neural network that takes as inputs: spatial coordinates (x̃, z̃), time (t̃), shear rate (γ̇), and concentration (c).

## Non-dimensionalized PDE

The governing equation is:

$$\frac{\partial c}{\partial \tilde{t}} + \tilde{u} \frac{\partial c}{\partial \tilde{x}} + \tilde{w} \frac{\partial c}{\partial \tilde{z}} + \frac{\partial}{\partial \tilde{z}} \left[ v_{seg}(\tilde{x}, \tilde{z}, \tilde{t}, \dot{\gamma}, c) \cdot c \right] = \frac{\partial}{\partial \tilde{z}} \left[ \frac{1}{Pe} \frac{\partial c}{\partial \tilde{z}} \right]$$

Expanding the segregation term:

$$\frac{\partial}{\partial \tilde{z}} \left[ v_{seg} \cdot c \right] = v_{seg} \frac{\partial c}{\partial \tilde{z}} + \frac{\partial v_{seg}}{\partial \tilde{z}} \cdot c$$

Where:
- $c$ is the concentration (volume fraction of small particles)
- $\tilde{x} \in [0, 1]$, $\tilde{z} \in [-1, 0]$, $\tilde{t} \in [0, t_{end}]$ are dimensionless coordinates
- $v_{seg}(\tilde{x}, \tilde{z}, \tilde{t}, \dot{\gamma}, c)$ is the **neural network** representing the segregation velocity (replaces $\Lambda(1-\tilde{x})g(\tilde{z})(1-c)$)
- $\dot{\gamma}$ is the shear rate (computed from $g(\tilde{z})$)
- $Pe$ is the Péclet number
- $\tilde{u}$ and $\tilde{w}$ are dimensionless velocity profiles

## Boundary Conditions

- **Inlet** ($\tilde{x} = 0$): Dirichlet $c = 0.5$ (well-mixed feed)
- **Top/Bottom** ($\tilde{z} = 0, -1$): Neumann $(1/Pe) \partial c/\partial \tilde{z} = v_{seg}(\tilde{x}, \tilde{z}, \tilde{t}, \dot{\gamma}, c) \cdot c$
- **Outlet** ($\tilde{x} = 1$): Natural boundary condition (zero diffusive flux)
- **Initial condition**: $c = 0.5$ everywhere at $\tilde{t} = 0$


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import grad
import time

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)


In [ ]:
# Parameters from the paper (section 3)
Pe = 4.0           # Péclet number
D = 1.0 / Pe        # vertical diffusivity
k = 2.3             # exponential velocity profile parameter
tEnd = 20.0         # final dimensionless time

# Domain bounds
x_min, x_max = 0.0, 1.0    # x̃ ∈ [0, 1]
z_min, z_max = -1.0, 0.0   # z̃ ∈ [-1, 0]
t_min, t_max = 0.0, tEnd   # t̃ ∈ [0, tEnd]

print(f"Parameters: Pe={Pe}, D={D:.6f}, k={k}")
print(f"Domain: x̃ ∈ [{x_min}, {x_max}], z̃ ∈ [{z_min}, {z_max}], t̃ ∈ [{t_min}, {t_max}]")


In [ ]:
# Helper functions for velocity profiles (u_tilde and w_tilde remain unchanged)
def u_tilde(x, z, k):
    """Dimensionless velocity in x direction"""
    # Convert k to tensor if it's not already a tensor
    if not isinstance(k, torch.Tensor):
        k = torch.tensor(float(k), dtype=x.dtype, device=x.device)
    factor = 0.5 * k * (1 - torch.exp(-k))
    return factor * (1 - x) * torch.exp(k * z)

def w_tilde(z, k):
    """Dimensionless velocity in z direction"""
    # Convert k to tensor if it's not already a tensor
    if not isinstance(k, torch.Tensor):
        k = torch.tensor(float(k), dtype=z.dtype, device=z.device)
    factor = 0.5 * (1 - torch.exp(-k))
    return factor * (torch.exp(k * z) - 1)

# Helper function to compute shear rate gamma_dot
def gamma_dot(z, k):
    """
    Compute shear rate gamma_dot from the shear-rate-like profile g(z).
    This is used as an input to the segregation velocity network.
    """
    if not isinstance(k, torch.Tensor):
        k = torch.tensor(float(k), dtype=z.dtype, device=z.device)
    g_val = (k**2 / (2 * (1 - torch.exp(-k)))) * torch.exp(k * z)
    return g_val


In [ ]:
# Neural Network Architecture for Segregation Velocity
class SegregationVelocityNet(nn.Module):
    """
    Neural network that learns the segregation velocity v_seg(x, z, t, gamma_dot, c).
    This replaces the functional form Λ(1-x)g(z)(1-c) from phase 2.1.
    Now parameterizes the entire segregation velocity including the (1-c) factor.
    """
    def __init__(self, layers):
        super(SegregationVelocityNet, self).__init__()
        self.layers = nn.ModuleList()
        
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))
        
        # Initialize weights using Xavier initialization
        for layer in self.layers:
            nn.init.xavier_uniform_(layer.weight)
            nn.init.zeros_(layer.bias)
    
    def forward(self, x, z, t, gamma_dot, c):
        """
        Forward pass: maps (x, z, t, gamma_dot, c) -> v_seg(x, z, t, gamma_dot, c)
        Args:
            x: x-coordinate tensor [batch_size, 1]
            z: z-coordinate tensor [batch_size, 1]
            t: t-coordinate tensor [batch_size, 1]
            gamma_dot: shear rate tensor [batch_size, 1]
            c: concentration tensor [batch_size, 1]
        Returns:
            v_seg: segregation velocity [batch_size, 1]
        """
        # Concatenate inputs: [x, z, t, gamma_dot, c]
        inputs = torch.cat([x, z, t, gamma_dot, c], dim=1)
        
        # Forward pass through hidden layers
        for i, layer in enumerate(self.layers[:-1]):
            inputs = torch.tanh(layer(inputs))
        
        # Output layer (no activation to allow positive and negative values)
        output = self.layers[-1](inputs)
        return output

# Main PINN Network for Concentration
class PINN(nn.Module):
    """
    Physics-Informed Neural Network that predicts concentration c(x, z, t).
    This network works together with SegregationVelocityNet to solve the PDE.
    """
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList()
        
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))
        
        # Initialize weights using Xavier initialization
        for layer in self.layers:
            nn.init.xavier_uniform_(layer.weight)
            nn.init.zeros_(layer.bias)
    
    def forward(self, x, z, t):
        """
        Forward pass: maps (x, z, t) -> c(x, z, t)
        Args:
            x: x-coordinate tensor [batch_size, 1]
            z: z-coordinate tensor [batch_size, 1]
            t: t-coordinate tensor [batch_size, 1]
        Returns:
            c: concentration [batch_size, 1], bounded between 0 and 1
        """
        # Concatenate inputs: [x, z, t]
        inputs = torch.cat([x, z, t], dim=1)
        
        # Forward pass through hidden layers
        for i, layer in enumerate(self.layers[:-1]):
            inputs = torch.tanh(layer(inputs))
        
        # Output layer with sigmoid activation to bound concentration between 0 and 1
        output = torch.sigmoid(self.layers[-1](inputs))
        return output

# Combined Model that includes both networks
class CombinedPINN(nn.Module):
    """
    Combined model that includes both the concentration network and segregation velocity network.
    This allows joint training of both networks.
    """
    def __init__(self, conc_layers, seg_layers):
        super(CombinedPINN, self).__init__()
        self.conc_net = PINN(conc_layers)
        self.seg_net = SegregationVelocityNet(seg_layers)
    
    def forward_conc(self, x, z, t):
        """Forward pass for concentration prediction"""
        return self.conc_net(x, z, t)
    
    def forward_seg(self, x, z, t, gamma_dot, c):
        """Forward pass for segregation velocity prediction"""
        return self.seg_net(x, z, t, gamma_dot, c)

# Network architectures
conc_layers = [3, 64, 64, 64, 64, 1]  # Concentration network: (x, z, t) -> c
seg_layers = [5, 32, 32, 32, 1]        # Segregation velocity network: (x, z, t, gamma_dot, c) -> v_seg

model = CombinedPINN(conc_layers, seg_layers).to(device)

print(f"Concentration network architecture: {conc_layers}")
print(f"Segregation velocity network architecture: {seg_layers}")
print(f"Total parameters (concentration): {sum(p.numel() for p in model.conc_net.parameters())}")
print(f"Total parameters (segregation): {sum(p.numel() for p in model.seg_net.parameters())}")
print(f"Total parameters (combined): {sum(p.numel() for p in model.parameters())}")


In [ ]:
def compute_pde_residual(x, z, t, model, Pe, k):
    """
    Compute the PDE residual using neural network for segregation velocity.
    
    PDE: ∂c/∂t̃ + ũ ∂c/∂x̃ + w̃ ∂c/∂z̃ + ∂/∂z̃ [v_seg(x,z,t,γ_dot,c) * c] = (1/Pe) ∂²c/∂z̃²
    
    The segregation term is now: ∂/∂z̃ [v_seg*c] = v_seg * ∂c/∂z̃ + (∂v_seg/∂z̃) * c
    
    Note: v_seg is parameterized as a neural network v_seg(x, z, t, gamma_dot, c),
    replacing the functional form Λ(1-x)g(z)(1-c) from phase 2.1.
    """
    # Clone and detach tensors to avoid modifying originals, then enable gradient computation
    x = x.clone().detach().requires_grad_(True)
    z = z.clone().detach().requires_grad_(True)
    t = t.clone().detach().requires_grad_(True)
    
    # Forward pass for concentration
    c = model.forward_conc(x, z, t)
    
    # Compute gradients of concentration
    c_t = grad(c, t, grad_outputs=torch.ones_like(c), create_graph=True)[0]
    c_x = grad(c, x, grad_outputs=torch.ones_like(c), create_graph=True)[0]
    c_z = grad(c, z, grad_outputs=torch.ones_like(c), create_graph=True)[0]
    
    # Second derivative in z
    c_zz = grad(c_z, z, grad_outputs=torch.ones_like(c_z), create_graph=True)[0]
    
    # Velocity profiles (advection)
    u_tilde_val = u_tilde(x, z, k)
    w_tilde_val = w_tilde(z, k)
    
    # Compute shear rate gamma_dot
    gamma_dot_val = gamma_dot(z, k)
    
    # Get segregation velocity from neural network: v_seg(x, z, t, gamma_dot, c)
    v_seg = model.forward_seg(x, z, t, gamma_dot_val, c)
    
    # Compute the segregation flux: v_seg * c
    seg_flux = v_seg * c
    
    # Compute derivative of segregation flux with respect to z: ∂/∂z [v_seg*c]
    seg_flux_z = grad(seg_flux, z, grad_outputs=torch.ones_like(seg_flux), create_graph=True)[0]
    
    # Diffusion term
    diff_term = (1.0 / Pe) * c_zz
    
    # Advection term
    adv_term = u_tilde_val * c_x + w_tilde_val * c_z
    
    # Segregation term: ∂/∂z [v_seg*c]
    seg_term = seg_flux_z
    
    # PDE residual: ∂c/∂t - (diffusion - advection - segregation)
    residual = c_t - (diff_term - adv_term - seg_term)
    
    return residual, c


In [ ]:
# Generate training points
def generate_training_data(n_pde, n_bc_inlet, n_bc_top, n_bc_bottom, n_bc_outlet, n_ic):
    """
    Generate collocation points for:
    - PDE residual points (interior)
    - Boundary condition points
    - Initial condition points
    """
    # PDE collocation points (interior domain)
    x_pde = torch.rand(n_pde, 1, device=device) * (x_max - x_min) + x_min
    z_pde = torch.rand(n_pde, 1, device=device) * (z_max - z_min) + z_min
    t_pde = torch.rand(n_pde, 1, device=device) * (t_max - t_min) + t_min
    
    # Boundary: x̃ = 0 (inlet, Dirichlet: c = 0.5)
    x_bc_inlet = torch.zeros(n_bc_inlet, 1, device=device)
    z_bc_inlet = torch.rand(n_bc_inlet, 1, device=device) * (z_max - z_min) + z_min
    t_bc_inlet = torch.rand(n_bc_inlet, 1, device=device) * (t_max - t_min) + t_min
    
    # Boundary: z̃ = 0 (top, Neumann)
    x_bc_top = torch.rand(n_bc_top, 1, device=device) * (x_max - x_min) + x_min
    z_bc_top = torch.zeros(n_bc_top, 1, device=device)
    t_bc_top = torch.rand(n_bc_top, 1, device=device) * (t_max - t_min) + t_min
    
    # Boundary: z̃ = -1 (bottom, Neumann)
    x_bc_bottom = torch.rand(n_bc_bottom, 1, device=device) * (x_max - x_min) + x_min
    z_bc_bottom = torch.ones(n_bc_bottom, 1, device=device) * z_min
    t_bc_bottom = torch.rand(n_bc_bottom, 1, device=device) * (t_max - t_min) + t_min
    
    # Boundary: x̃ = 1 (outlet, zero diffusive flux - natural BC, handled by PDE)
    x_bc_outlet = torch.ones(n_bc_outlet, 1, device=device)
    z_bc_outlet = torch.rand(n_bc_outlet, 1, device=device) * (z_max - z_min) + z_min
    t_bc_outlet = torch.rand(n_bc_outlet, 1, device=device) * (t_max - t_min) + t_min
    
    # Initial condition: t̃ = 0, c = 0.5 everywhere
    x_ic = torch.rand(n_ic, 1, device=device) * (x_max - x_min) + x_min
    z_ic = torch.rand(n_ic, 1, device=device) * (z_max - z_min) + z_min
    t_ic = torch.zeros(n_ic, 1, device=device)
    
    return {
        'pde': (x_pde, z_pde, t_pde),
        'bc_inlet': (x_bc_inlet, z_bc_inlet, t_bc_inlet),
        'bc_top': (x_bc_top, z_bc_top, t_bc_top),
        'bc_bottom': (x_bc_bottom, z_bc_bottom, t_bc_bottom),
        'bc_outlet': (x_bc_outlet, z_bc_outlet, t_bc_outlet),
        'ic': (x_ic, z_ic, t_ic)
    }

# Number of training points
n_pde = 10000
n_bc_inlet = 1000
n_bc_top = 1000
n_bc_bottom = 1000
n_bc_outlet = 1000
n_ic = 2000

train_data = generate_training_data(n_pde, n_bc_inlet, n_bc_top, n_bc_bottom, n_bc_outlet, n_ic)
print("Training data generated:")
for key, (x, z, t) in train_data.items():
    print(f"  {key}: {x.shape[0]} points")


In [ ]:
def compute_loss(model, train_data, Pe, k, weights, exp_data=None):
    """
    Compute total loss using neural network for segregation velocity.
    L = w_pde * L_pde + w_bc_inlet * L_bc_inlet + w_bc_top * L_bc_top 
        + w_bc_bottom * L_bc_bottom + w_ic * L_ic + w_data * L_data
    
    Args:
        model: CombinedPINN model (contains both concentration and segregation networks)
        train_data: Dictionary of training collocation points
        Pe: Péclet number
        k: Velocity profile parameter
        weights: Dictionary of loss weights
        exp_data: Optional experimental data dictionary with keys:
            'x', 'z', 't', 'c' (concentration measurements)
    """
    total_loss = 0.0
    
    # PDE residual loss
    x_pde, z_pde, t_pde = train_data['pde']
    residual, _ = compute_pde_residual(x_pde, z_pde, t_pde, model, Pe, k)
    loss_pde = torch.mean(residual**2)
    total_loss += weights['pde'] * loss_pde
    
    # Boundary condition: x̃ = 0, c = 0.5 (Dirichlet)
    x_in, z_in, t_in = train_data['bc_inlet']
    c_in = model.forward_conc(x_in, z_in, t_in)
    loss_bc_inlet = torch.mean((c_in - 0.5)**2)
    total_loss += weights['bc_inlet'] * loss_bc_inlet
    
    # Boundary condition: z̃ = 0 (top, Neumann: (1/Pe) ∂c/∂z = v_seg(x,z,t,γ_dot,c) * c)
    x_top, z_top, t_top = train_data['bc_top']
    x_top = x_top.clone().detach().requires_grad_(True)
    z_top = z_top.clone().detach().requires_grad_(True)
    t_top = t_top.clone().detach()
    c_top = model.forward_conc(x_top, z_top, t_top)
    c_z_top = grad(c_top, z_top, grad_outputs=torch.ones_like(c_top), create_graph=True)[0]
    gamma_dot_top = gamma_dot(z_top, k)
    v_seg_top = model.forward_seg(x_top, z_top, t_top, gamma_dot_top, c_top)
    bc_top_target = v_seg_top * c_top  # Segregation flux at boundary
    bc_top_pred = (1.0 / Pe) * c_z_top  # Diffusive flux
    loss_bc_top = torch.mean((bc_top_pred - bc_top_target)**2)
    total_loss += weights['bc_top'] * loss_bc_top
    
    # Boundary condition: z̃ = -1 (bottom, Neumann: (1/Pe) ∂c/∂z = v_seg(x,z,t,γ_dot,c) * c)
    x_bot, z_bot, t_bot = train_data['bc_bottom']
    x_bot = x_bot.clone().detach().requires_grad_(True)
    z_bot = z_bot.clone().detach().requires_grad_(True)
    t_bot = t_bot.clone().detach()
    c_bot = model.forward_conc(x_bot, z_bot, t_bot)
    c_z_bot = grad(c_bot, z_bot, grad_outputs=torch.ones_like(c_bot), create_graph=True)[0]
    gamma_dot_bot = gamma_dot(z_bot, k)
    v_seg_bot = model.forward_seg(x_bot, z_bot, t_bot, gamma_dot_bot, c_bot)
    bc_bot_target = v_seg_bot * c_bot  # Segregation flux at boundary
    bc_bot_pred = (1.0 / Pe) * c_z_bot  # Diffusive flux
    loss_bc_bottom = torch.mean((bc_bot_pred - bc_bot_target)**2)
    total_loss += weights['bc_bottom'] * loss_bc_bottom
    
    # Initial condition: t̃ = 0, c = 0.5
    x_ic, z_ic, t_ic = train_data['ic']
    c_ic = model.forward_conc(x_ic, z_ic, t_ic)
    loss_ic = torch.mean((c_ic - 0.5)**2)
    total_loss += weights['ic'] * loss_ic
    
    # Experimental data loss (if provided)
    loss_data = torch.tensor(0.0, device=device)
    if exp_data is not None:
        x_data = exp_data['x']
        z_data = exp_data['z']
        t_data = exp_data['t']
        c_data = exp_data['c']  # Measured concentration values
        
        # Predict concentration at experimental points
        c_pred = model.forward_conc(x_data, z_data, t_data)
        loss_data = torch.mean((c_pred - c_data)**2)
        total_loss += weights.get('data', 1.0) * loss_data
    
    # Outlet boundary (x̃ = 1): zero diffusive flux is naturally satisfied
    # by the PDE, so we don't need to enforce it explicitly
    
    loss_dict = {
        'pde': loss_pde.item(),
        'bc_inlet': loss_bc_inlet.item(),
        'bc_top': loss_bc_top.item(),
        'bc_bottom': loss_bc_bottom.item(),
        'ic': loss_ic.item()
    }
    
    if exp_data is not None:
        loss_dict['data'] = loss_data.item()
    
    return total_loss, loss_dict


In [ ]:
# ============================================================================
# STEP 1: Prepare Experimental Data (if available)
# ============================================================================
# If you have experimental data, load it here and format it as tensors.
# The data should contain: x, z, t (coordinates) and c (concentration measurements)

use_experimental_data = True  # Set to True when you have experimental data

if use_experimental_data:
    # Load from CSV file
    import pandas as pd
    data = pd.read_csv('experimental_data.csv')
    x_exp = torch.tensor(data['x'].values, dtype=torch.float32, device=device).reshape(-1, 1)
    z_exp = torch.tensor(data['z'].values, dtype=torch.float32, device=device).reshape(-1, 1)
    t_exp = torch.tensor(data['t'].values, dtype=torch.float32, device=device).reshape(-1, 1)
    c_exp = torch.tensor(data['c'].values, dtype=torch.float32, device=device).reshape(-1, 1)
    n_exp = x_exp.shape[0]
    
    exp_data = {
        'x': x_exp,
        'z': z_exp,
        't': t_exp,
        'c': c_exp
    }
    print(f"Loaded {n_exp} experimental data points")
else:
    exp_data = None
    print("No experimental data provided. Both networks will be learned from physics constraints only.")

# ============================================================================
# STEP 2: Training Setup
# ============================================================================
# Both networks (concentration and segregation velocity) will be trained jointly
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1000)

# Loss weights (can be adjusted)
# Increase 'data' weight if you have experimental data to emphasize data fitting
loss_weights = {
    'pde': 1.0,
    'bc_inlet': 10.0,
    'bc_top': 10.0,
    'bc_bottom': 10.0,
    'ic': 10.0,
    'data': 50.0 if use_experimental_data else 0.0  # Higher weight for experimental data
}

# Training parameters
n_epochs = 20000
print_interval = 500
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs) # option 2: cosine annealing

# Training loop
loss_history = []
start_time = time.time()

print("Starting training...")
print("Training both concentration network and segregation velocity network jointly.")
if use_experimental_data:
    print(f"{'Epoch':<10} {'Total Loss':<15} {'PDE':<15} {'BC Inlet':<15} {'BC Top':<15} {'BC Bottom':<15} {'IC':<15} {'Data':<15}")
    print("-" * 120)
else:
    print(f"{'Epoch':<10} {'Total Loss':<15} {'PDE':<15} {'BC Inlet':<15} {'BC Top':<15} {'BC Bottom':<15} {'IC':<15}")
    print("-" * 120)

try:
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        
        # Compute loss
        total_loss, loss_dict = compute_loss(model, train_data, Pe, k, loss_weights, exp_data=exp_data)
        
        # Backward pass
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
#         scheduler.step(total_loss)
        scheduler.step()
        
        # Store loss history
        loss_history.append({
            'epoch': epoch,
            'total': total_loss.item(),
            **loss_dict
        })
        
        # Print progress
        if (epoch + 1) % print_interval == 0 or epoch == 0:
            if use_experimental_data:
                print(f"{epoch+1:<10} {total_loss.item():<15.6e} {loss_dict['pde']:<15.6e} "
                      f"{loss_dict['bc_inlet']:<15.6e} {loss_dict['bc_top']:<15.6e} "
                      f"{loss_dict['bc_bottom']:<15.6e} {loss_dict['ic']:<15.6e} "
                      f"{loss_dict.get('data', float('nan')):<15.6f}")
            else:
                print(f"{epoch+1:<10} {total_loss.item():<15.6e} {loss_dict['pde']:<15.6e} "
                      f"{loss_dict['bc_inlet']:<15.6e} {loss_dict['bc_top']:<15.6e} "
                      f"{loss_dict['bc_bottom']:<15.6e} {loss_dict['ic']:<15.6e}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user.")
except Exception as e:
    print(f"\nError during training: {e}")
    import traceback
    traceback.print_exc()

elapsed_time = time.time() - start_time
print(f"\nTraining completed in {elapsed_time:.2f} seconds")


In [ ]:
# Plot loss history
if len(loss_history) == 0:
    print("Warning: loss_history is empty. Please run the training cell first.")
else:
    if use_experimental_data:
        loss_history_array = np.array([(h['epoch'], h['total'], h['pde'], h['bc_inlet'], 
                                    h['bc_top'], h['bc_bottom'], h['ic'], h['data']) for h in loss_history])
    else:
        loss_history_array = np.array([(h['epoch'], h['total'], h['pde'], h['bc_inlet'], 
                                    h['bc_top'], h['bc_bottom'], h['ic']) for h in loss_history])
    
    fig, axes = plt.subplots(2, 1, figsize=(10, 8))
    
    # Total loss
    axes[0].semilogy(loss_history_array[:, 0], loss_history_array[:, 1], 'b-', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Total Loss', fontsize=12)
    axes[0].set_title('Total Loss History', fontsize=14)
    axes[0].grid(True, alpha=0.3)
    if 'n_epochs' in globals():
        axes[0].set_xlim([0, n_epochs])
    else:
        axes[0].set_xlim([0, loss_history_array[-1, 0] + 1])
    
    # Individual losses
    axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 2], 'r-', label='PDE', linewidth=2)
    axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 3], 'g-', label='BC Inlet', linewidth=2)
    axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 4], 'm-', label='BC Top', linewidth=2)
    axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 5], 'c-', label='BC Bottom', linewidth=2)
    axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 6], 'y-', label='IC', linewidth=2)
    if use_experimental_data:
        axes[1].semilogy(loss_history_array[:, 0], loss_history_array[:, 7], 'k-', label='Data', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].set_title('Individual Loss Components', fontsize=14)
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)
    if 'n_epochs' in globals():
        axes[1].set_xlim([0, n_epochs])
    else:
        axes[1].set_xlim([0, loss_history_array[-1, 0] + 1])
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Visualize the learned segregation velocity network v_seg(x, z)
model.eval()
with torch.no_grad():
    # Create grid for visualization
    n_x, n_z = 100, 100
    x_vis = torch.linspace(x_min, x_max, n_x, device=device).reshape(-1, 1)
    z_vis = torch.linspace(z_min, z_max, n_z, device=device).reshape(-1, 1)
    X_vis, Z_vis = torch.meshgrid(x_vis.squeeze(), z_vis.squeeze(), indexing='ij')
    
    # Evaluate segregation velocity network at final time
    # Need: x, z, t, gamma_dot, c
    t_vis = torch.ones_like(X_vis.reshape(-1, 1)) * t_max
    gamma_dot_vis = gamma_dot(Z_vis.reshape(-1, 1), k)
    c_vis = model.forward_conc(X_vis.reshape(-1, 1), Z_vis.reshape(-1, 1), t_vis)
    
    # Evaluate segregation velocity network: v_seg(x, z, t, gamma_dot, c)
    v_seg_vis = model.forward_seg(X_vis.reshape(-1, 1), Z_vis.reshape(-1, 1), 
                                   t_vis, gamma_dot_vis, c_vis)
    V_seg_vis = v_seg_vis.reshape(n_x, n_z).cpu().numpy()
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), V_seg_vis, levels=50, cmap='viridis')
    ax.set_xlabel(r'$\tilde{x}$', fontsize=14)
    ax.set_ylabel(r'$\tilde{z}$', fontsize=14)
    ax.set_title('Learned Segregation Velocity $v_{seg}(\\tilde{x}, \\tilde{z}, \\tilde{t}, \\gamma_dot, c)$ from Neural Network', fontsize=14)
    cbar = plt.colorbar(im, ax=ax, label='$v_{seg}$')
    ax.set_aspect('equal')
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"Segregation velocity statistics:")
    print(f"  Mean: {np.mean(V_seg_vis):.6f}")
    print(f"  Std:  {np.std(V_seg_vis):.6f}")
    print(f"  Min:  {np.min(V_seg_vis):.6f}")
    print(f"  Max:  {np.max(V_seg_vis):.6f}")


In [ ]:
# Visualize solution (concentration) at final time
model.eval()
with torch.no_grad():
    # Create grid for visualization
    n_x, n_z = 100, 100
    x_vis = torch.linspace(x_min, x_max, n_x, device=device).reshape(-1, 1)
    z_vis = torch.linspace(z_min, z_max, n_z, device=device).reshape(-1, 1)
    X_vis, Z_vis = torch.meshgrid(x_vis.squeeze(), z_vis.squeeze(), indexing='ij')
    
    # Evaluate at final time
    t_final = torch.ones_like(X_vis.reshape(-1, 1)) * t_max
    c_final = model.forward_conc(X_vis.reshape(-1, 1), Z_vis.reshape(-1, 1), t_final)
    C_final = c_final.reshape(n_x, n_z).cpu().numpy()
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))
    levels = np.linspace(0.0, 1.0, 51)
    im = ax.contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), 1 - C_final, levels=levels, cmap='hot')
    ax.set_xlabel(r'$\tilde{x}$', fontsize=14)
    ax.set_ylabel(r'$\tilde{z}$', fontsize=14)
    ax.set_title(f'Concentration $c_s$ at $\\tilde{{t}}={t_max}$ (Pe={Pe})', fontsize=14)
    cbar = plt.colorbar(im, ax=ax, label='Concentration $c_s$')
    ax.set_aspect('equal')
    plt.tight_layout()
    plt.show()


In [ ]:
# Visualize solution evolution over time
model.eval()
with torch.no_grad():
    n_x, n_z = 100, 100
    x_vis = torch.linspace(x_min, x_max, n_x, device=device).reshape(-1, 1)
    z_vis = torch.linspace(z_min, z_max, n_z, device=device).reshape(-1, 1)
    X_vis, Z_vis = torch.meshgrid(x_vis.squeeze(), z_vis.squeeze(), indexing='ij')
    
    # Plot at different times
    times = [0.0, 2.5, 5.0, 7.5, 10.0]
    levels = np.linspace(0.0, 1.0, 51)
    fig, axes = plt.subplots(1, len(times), figsize=(20, 4))
    
    for idx, t_val in enumerate(times):
        t_vis = torch.ones_like(X_vis.reshape(-1, 1)) * t_val
        c_vis = model.forward_conc(X_vis.reshape(-1, 1), Z_vis.reshape(-1, 1), t_vis)
        C_vis = c_vis.reshape(n_x, n_z).cpu().numpy()
        
        im = axes[idx].contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), 1-C_vis, levels=levels, cmap='hot')
        im.set_clim(0.0, 1.0)  # Explicitly set color limits
        axes[idx].set_xlabel(r'$\tilde{x}$', fontsize=12)
        axes[idx].set_ylabel(r'$\tilde{z}$', fontsize=12)
        axes[idx].set_title(f'$\\tilde{{t}}={t_val}$', fontsize=12)
        axes[idx].set_aspect('equal')
        cbar = plt.colorbar(im, ax=axes[idx])
    
    plt.suptitle(f'Concentration Evolution (Pe={Pe})', fontsize=14)
    plt.tight_layout()
    plt.show()


In [ ]:
# # Generate movie of solution evolution over time
# from matplotlib.animation import FuncAnimation, FFMpegWriter
# import matplotlib

# model.eval()
# with torch.no_grad():
#     # Create grid for visualization
#     n_x, n_z = 100, 100
#     x_vis = torch.linspace(x_min, x_max, n_x, device=device).reshape(-1, 1)
#     z_vis = torch.linspace(z_min, z_max, n_z, device=device).reshape(-1, 1)
#     X_vis, Z_vis = torch.meshgrid(x_vis.squeeze(), z_vis.squeeze(), indexing='ij')
    
#     # Time points for animation
#     n_frames = 100
#     time_points = np.linspace(t_min, t_max, n_frames)
    
#     # Pre-compute all frames
#     print("Pre-computing frames for animation...")
#     frames = []
#     for i, t_val in enumerate(time_points):
#         t_vis = torch.ones_like(X_vis.reshape(-1, 1)) * t_val
#         c_vis = model.forward_conc(X_vis.reshape(-1, 1), Z_vis.reshape(-1, 1), t_vis)
#         C_vis = c_vis.reshape(n_x, n_z).cpu().numpy()
#         frames.append(1 - C_vis)  # Convert to c_s (small particle concentration)
#         if (i + 1) % 20 == 0:
#             print(f"  Processed {i + 1}/{n_frames} frames")
    
#     # Create figure and axis
#     fig, ax = plt.subplots(figsize=(10, 8))
#     levels = np.linspace(0.0, 1.0, 51)
    
#     # Initial frame
#     im = ax.contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), frames[0], 
#                      levels=levels, cmap='hot')
#     im.set_clim(0.0, 1.0)
#     ax.set_xlabel(r'$\tilde{x}$', fontsize=14)
#     ax.set_ylabel(r'$\tilde{z}$', fontsize=14)
#     ax.set_aspect('equal')
#     cbar = plt.colorbar(im, ax=ax, label='Concentration $c_s$')
#     title = ax.set_title(f'Concentration Evolution: $\\tilde{{t}}={time_points[0]:.2f}$ (Pe={Pe})', 
#                          fontsize=14)
    
#     # Animation update function
#     def animate(frame_idx):
#         # Clear only the contour plot and title, keep axes and colorbar
#         for collection in ax.collections:
#             collection.remove()
#         ax.set_title(f'Concentration Evolution: $\\tilde{{t}}={time_points[frame_idx]:.2f}$ (Pe={Pe})', 
#                      fontsize=14)
#         # Create new contour plot
#         im = ax.contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), frames[frame_idx], 
#                          levels=levels, cmap='hot')
#         im.set_clim(0.0, 1.0)
#         return im.collections
    
#     # Create animation
#     print("Creating animation...")
#     anim = FuncAnimation(fig, animate, frames=n_frames, interval=100, blit=False, repeat=True)
    
#     # Save as movie file
#     print("Saving movie...")
#     try:
#         # Try using FFMpegWriter (requires ffmpeg to be installed)
#         writer = FFMpegWriter(fps=10, metadata=dict(artist='PINN'), bitrate=1800)
#         anim.save('solution_evolution_phase2.2.mp4', writer=writer)
#         print("Movie saved as 'solution_evolution_phase2.2.mp4'")
#     except Exception as e:
#         print(f"Error saving with FFMpegWriter: {e}")
#         print("Trying alternative method with PillowWriter...")
#         try:
#             from matplotlib.animation import PillowWriter
#             writer = PillowWriter(fps=10)
#             anim.save('solution_evolution_phase2.2.gif', writer=writer)
#             print("Movie saved as 'solution_evolution_phase2.2.gif'")
#         except Exception as e2:
#             print(f"Error saving with PillowWriter: {e2}")
#             print("Displaying animation inline instead...")
#             plt.show()
    
#     # Also display the animation inline
#     plt.tight_layout()
#     plt.show()


In [ ]:
# Check PDE residual at final time
model.eval()  # Set to eval mode (but we still need gradients for residual computation)
# Sample points in the domain
n_check = 1000
x_check = torch.rand(n_check, 1, device=device) * (x_max - x_min) + x_min
z_check = torch.rand(n_check, 1, device=device) * (z_max - z_min) + z_min
t_check = torch.ones(n_check, 1, device=device) * t_max

# Enable gradients for residual computation
x_check.requires_grad_(True)
z_check.requires_grad_(True)
t_check.requires_grad_(True)

residual, c_check = compute_pde_residual(x_check, z_check, t_check, model, Pe, k)

# Detach for statistics (no need for gradients)
residual_detached = residual.detach()

print(f"PDE Residual Statistics at t={t_max}:")
print(f"  Mean: {torch.mean(torch.abs(residual_detached)).item():.6e}")
print(f"  Std:  {torch.std(residual_detached).item():.6e}")
print(f"  Max:  {torch.max(torch.abs(residual_detached)).item():.6e}")
print(f"  Min:  {torch.min(torch.abs(residual_detached)).item():.6e}")

# Plot residual distribution
fig, ax = plt.subplots(figsize=(8, 6))
ax.hist(residual_detached.cpu().numpy().flatten(), bins=50, edgecolor='black', alpha=0.7)
ax.set_xlabel('PDE Residual', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Distribution of PDE Residuals at Final Time', fontsize=14)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Optional: Compare learned segregation velocity with functional form from phase 2.1
# This requires loading the model from phase 2.1 or having Lambda and g(z) parameters

# For comparison, let's create a functional form based on typical values
# You can replace this with actual values from phase 2.1 if available
Lambda_comparison = 0.78  # Example value from phase 2.1

def g_profile_comparison(z, k):
    """Shear-rate-like profile g(z̃) for comparison"""
    if not isinstance(k, torch.Tensor):
        k = torch.tensor(float(k), dtype=z.dtype, device=z.device)
    g_val = (k**2 / (2 * (1 - torch.exp(-k)))) * torch.exp(k * z)
    return g_val

model.eval()
with torch.no_grad():
    # Create grid for comparison
    n_x, n_z = 100, 100
    x_vis = torch.linspace(x_min, x_max, n_x, device=device).reshape(-1, 1)
    z_vis = torch.linspace(z_min, z_max, n_z, device=device).reshape(-1, 1)
    X_vis, Z_vis = torch.meshgrid(x_vis.squeeze(), z_vis.squeeze(), indexing='ij')
    
    # Evaluate at final time for comparison
    t_vis = torch.ones_like(X_vis.reshape(-1, 1)) * t_max
    gamma_dot_vis = gamma_dot(Z_vis.reshape(-1, 1), k)
    c_vis = model.forward_conc(X_vis.reshape(-1, 1), Z_vis.reshape(-1, 1), t_vis)
    
    # Learned segregation velocity from neural network: v_seg(x, z, t, gamma_dot, c)
    v_seg_nn = model.forward_seg(X_vis.reshape(-1, 1), Z_vis.reshape(-1, 1), 
                                  t_vis, gamma_dot_vis, c_vis)
    V_seg_nn = v_seg_nn.reshape(n_x, n_z).cpu().numpy()
    
    # Functional form from phase 2.1: Lambda * (1 - x) * g(z) * (1 - c)
    # Note: In phase 2.2, we parameterize the entire v_seg including (1-c) factor
    g_val = g_profile_comparison(Z_vis.reshape(-1, 1), k)
    v_seg_func = Lambda_comparison * (1 - X_vis.reshape(-1, 1)) * g_val * (1 - c_vis.reshape(-1, 1))
    V_seg_func = v_seg_func.reshape(n_x, n_z).cpu().numpy()
    
    # Difference
    V_seg_diff = V_seg_nn - V_seg_func
    
    # Create comparison plot
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Learned neural network
    levels = np.linspace(V_seg_nn.min(), V_seg_nn.max(), 50)
    im1 = axes[0].contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), V_seg_nn, cmap='viridis', levels=levels)
    axes[0].set_xlabel(r'$\tilde{x}$', fontsize=12)
    axes[0].set_ylabel(r'$\tilde{z}$', fontsize=12)
    axes[0].set_title('Learned Neural Network', fontsize=12)
    axes[0].set_aspect('equal')
    plt.colorbar(im1, ax=axes[0])
    
    # Functional form (from phase 2.1)
    im2 = axes[1].contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), V_seg_func, cmap='viridis', levels=levels)
    axes[1].set_xlabel(r'$\tilde{x}$', fontsize=12)
    axes[1].set_ylabel(r'$\tilde{z}$', fontsize=12)
    axes[1].set_title(f'Functional Form (Λ={Lambda_comparison:.2f})', fontsize=12)
    axes[1].set_aspect('equal')
    plt.colorbar(im2, ax=axes[1])
    
    # Difference
    im3 = axes[2].contourf(X_vis.cpu().numpy(), Z_vis.cpu().numpy(), V_seg_diff, levels=50, cmap='RdBu_r')
    axes[2].set_xlabel(r'$\tilde{x}$', fontsize=12)
    axes[2].set_ylabel(r'$\tilde{z}$', fontsize=12)
    axes[2].set_title('Difference (NN - Functional)', fontsize=12)
    axes[2].set_aspect('equal')
    plt.colorbar(im3, ax=axes[2])
    
    plt.suptitle('Comparison: Neural Network vs Functional Form for Segregation Velocity', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print(f"Difference statistics:")
    print(f"  Mean: {np.mean(V_seg_diff):.6f}")
    print(f"  Std:  {np.std(V_seg_diff):.6f}")
    print(f"  Max:  {np.max(V_seg_diff):.6f}")
    print(f"  Min:  {np.min(V_seg_diff):.6f}")


In [ ]:
# Compute and plot residuals between PINN solution and experimental data (if available)
if exp_data is None:
    print("No experimental data available. Cannot compute residuals.")
else:
    model.eval()
    with torch.no_grad():
        # Get experimental data
        x_exp = exp_data['x']
        z_exp = exp_data['z']
        t_exp = exp_data['t']
        c_exp = exp_data['c']  # Actual experimental measurements
        
        # Predict concentrations at experimental points
        c_pred = model.forward_conc(x_exp, z_exp, t_exp)
        
        # Compute residuals (predicted - actual)
        residuals = (c_pred - c_exp).cpu().numpy().flatten()
        c_pred_np = c_pred.cpu().numpy().flatten()
        c_exp_np = c_exp.cpu().numpy().flatten()
        
        # Statistics
        mean_residual = np.mean(residuals)
        std_residual = np.std(residuals)
        mse = np.mean(residuals**2)
        rmse = np.sqrt(mse)
        mae = np.mean(np.abs(residuals))
        
        print("Residual Statistics:")
        print(f"  Mean residual: {mean_residual:.6e}")
        print(f"  Std residual:  {std_residual:.6e}")
        print(f"  MSE:           {mse:.6e}")
        print(f"  RMSE:          {rmse:.6e}")
        print(f"  MAE:           {mae:.6e}")
        
        # Scatter plot: Predicted vs Actual
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.scatter(c_exp_np, c_pred_np, alpha=0.6, s=20, color='steelblue')
        # Perfect prediction line (y=x)
        min_val = min(c_exp_np.min(), c_pred_np.min())
        max_val = max(c_exp_np.max(), c_pred_np.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect prediction')
        ax.set_xlabel('Actual Concentration', fontsize=12)
        ax.set_ylabel('Predicted Concentration', fontsize=12)
        ax.set_title('Predicted vs Actual Concentration', fontsize=14)
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal', adjustable='box')
        plt.tight_layout()
        plt.show()


In [ ]:
# Save the model
file_name = './models/pinn_inverse_NN.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'conc_layers': conc_layers,
    'seg_layers': seg_layers,
    'Pe': Pe,
    'k': k,
    'tEnd': tEnd,
    'loss_history': loss_history,
    'x_min': x_min, 'x_max': x_max,
    'z_min': z_min, 'z_max': z_max,
    't_min': t_min, 't_max': t_max,
    'pde points': n_pde,
    'bc_inlet points': n_bc_inlet,
    'bc_top points': n_bc_top,
    'bc_bottom points': n_bc_bottom,
    'bc_outlet points': n_bc_outlet,
    'ic points': n_ic,
    'exp points': n_exp
}, file_name)
print(f"Model saved to '{file_name}'")
